In [5]:
import pandas as pd
from omegaconf import OmegaConf
from lib.uncertainty import Simulate
from prettytable import PrettyTable

In [6]:
EXPERIMENTS = [
    '2026-01-20_11-24' # Oracle
]

strategy = {
    'oracle': 'Oracle',
    'random': 'Random',
    'similar': 'Similar',
    'dense': 'Dense'
}

In [7]:
results_all = {}

for experiment in EXPERIMENTS:
    cfg = OmegaConf.load(f'../results/{experiment}/config.yaml')
    results = pd.read_json(f'../results/{experiment}/results.json')
    results['correct_query'] = True
    results['retriever_strategy'] = cfg.retriever.strategy

    results_all[cfg.retriever.strategy] = results

In [8]:
simulate = Simulate(cfg)

t = PrettyTable(field_names=['Retriever Strategy', 'Query', 'Retreiver', 'Generator', 'Rows'])

for experiment, results in results_all.items():
    success_rates = simulate.compute_uncertainty(results)

    t.add_row([
        strategy[experiment],
        f"{success_rates['q']['mean']:.2%} ± {success_rates['q']['std']:.4%}",
        f"{success_rates['r']['mean']:.2%} ± {success_rates['r']['std']:.4%}",
        f"{success_rates['g']['mean']:.2%} ± {success_rates['g']['std']:.4%}",
        len(results)
    ])


t

Retriever Strategy,Query,Retreiver,Generator,Rows
Dense,99.99% ± 0.0099%,65.59% ± 0.4775%,78.61% ± 0.4093%,10000
